### Problem 1

Calculate and compare the expected value and standard deviation of the price at time \( P_t \), given each of the 3 types of price returns, assuming 

$$ r_t \sim N(0, \sigma^2 = 0.01) $$

and the price at time \( t-1 \) \( P_{t-1} \).

Simulate each return equation using $$ r_t \sim N(0, \sigma^2) $$ and show the mean and standard deviation match your expectations.

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import t
from scipy.stats import norm
from statsmodels.tsa.ar_model import AutoReg

In [3]:
# 调整 pandas 显示
pd.set_option('display.width', 500)  

In [4]:
# 生成来自正态分布的收益率 rt​ ∼N(0,0.01)。
# 设置参数
sigma = 0.1  # 标准差 sqrt(0.01)
n = 1000  # 模拟次数
P_t_minus_1 = 100  # 初始价格

# 生成正态分布的return
returns = np.random.normal(0, sigma, n)

# 前10个return
print(returns[:10])


[-0.22168128  0.05519064  0.01553907  0.08564817 -0.02336335  0.06978542
 -0.01192886 -0.07571657 -0.01787045 -0.00944135]


In [5]:
# 经典布朗运动计算
P_brownian = P_t_minus_1 + returns

# 算术收益系统计算
P_arithmetic = P_t_minus_1 * (1 + returns)

# 对数收益（几何布朗运动）计算
P_log_return = P_t_minus_1 * np.exp(returns)

In [6]:
# 将价格数据存入df
df_prices = pd.DataFrame({
    'Returns': returns,
    'Classical Brownian Motion': P_brownian,
    'Arithmetic Return': P_arithmetic,
    'Log Return (Geometric Brownian)': P_log_return
})

print(df_prices)

      Returns  Classical Brownian Motion  Arithmetic Return  Log Return (Geometric Brownian)
0   -0.221681                  99.778319          77.831872                        80.117067
1    0.055191                 100.055191         105.519064                       105.674205
2    0.015539                 100.015539         101.553907                       101.566043
3    0.085648                 100.085648         108.564817                       108.942297
4   -0.023363                  99.976637          97.663665                        97.690746
..        ...                        ...                ...                              ...
995  0.046554                 100.046554         104.655420                       104.765486
996 -0.007920                  99.992080          99.208044                        99.211172
997 -0.192157                  99.807843          80.784318                        82.517746
998 -0.122930                  99.877070          87.706954           

In [7]:
# 计算每种方法的均值和标准差
mean_brownian = np.mean(P_brownian)
std_brownian = np.std(P_brownian)

mean_arithmetic = np.mean(P_arithmetic)
std_arithmetic = np.std(P_arithmetic)

mean_log_return = np.mean(P_log_return)
std_log_return = np.std(P_log_return)

In [8]:
# 将均值和标准差添加到df
df_results = pd.DataFrame({
    'Model': ['Classical Brownian Motion', 'Arithmetic Return', 'Log Return (Geometric Brownian)'],
    'Mean': [mean_brownian, mean_arithmetic, mean_log_return],
    'Standard Deviation': [std_brownian, std_arithmetic, std_log_return]
})

# Price的前几行
print("Price Data：")
print(df_prices.head())

# return的均值和标准差
print("\nMean & Standard Deviation of returns：")
print(df_results)

Price Data：
    Returns  Classical Brownian Motion  Arithmetic Return  Log Return (Geometric Brownian)
0 -0.221681                  99.778319          77.831872                        80.117067
1  0.055191                 100.055191         105.519064                       105.674205
2  0.015539                 100.015539         101.553907                       101.566043
3  0.085648                 100.085648         108.564817                       108.942297
4 -0.023363                  99.976637          97.663665                        97.690746

Mean & Standard Deviation of returns：
                             Model        Mean  Standard Deviation
0        Classical Brownian Motion   99.997364            0.101067
1                Arithmetic Return   99.736390           10.106721
2  Log Return (Geometric Brownian)  100.248524           10.191437


**经典布朗运动**（Classical Brownian Motion）
- **理论预期**：由于收益率r_t~N(0, 0.01)，价格 P_t应该围绕初始价格 P_(t-1)(假设为100)波动。期望值应接近初始价格 100，标准差应接近收益率的标准差0.1。
- **模拟结果**：
  - **均值**：99.999740，几乎与 100 完全一致。
  - **标准差**：0.099464，接近 0.1。
- **结论**：结果符合理论预期。经典布朗运动模型简单地加上了正态分布的随机收益率，保持价格的波动较小，符合预期的均值和标准差。

**算术收益系统**（Arithmetic Return System）
- **理论预期**：算术收益系统中，价格通过P_t = P_(t-1)(1 + r_t)计算。由于是乘法关系，价格波动应该比经典布朗运动更大，尤其是当收益率波动较大时，标准差会更高。
- **模拟结果**：
  - **均值**：99.974048，接近初始价格 100。
  - **标准差**：9.946425，明显大于经典布朗运动的标准差。
- **结论**：结果符合理论预期。虽然均值接近 100，但由于价格的变化是按比例计算的，波动率被放大，因此标准差比经典布朗运动大很多，即算术收益模型表现出较大的波动性。

**对数收益/几何布朗运动**（Log Return/Geometric Brownian Motion）
- **理论预期**：对数收益系统P_t = P_(t-1)e^(r_t)通过指数函数计算价格变化，复合增长的特性会导致较大的价格波动。
- **模拟结果**：
  - **均值**：100.469299，比初始价格略高。
  - **标准差**：10.002250，波动性最大。
- **结论**：结果符合理论预期。由于指数增长，价格在有正收益时会快速上升，均值略高于 100。标准差也较大，这反映了复合收益的累积效应，因此波动性最高。


**结论：通过比较三种模型的均值和标准差，所有结果都与理论预期一致**

### Problem 2  
Implement a function similar to the “return_calculate()” in this week’s code. Allow the user to specify the method of return calculation.  
Use DailyPrices.csv. Calculate the arithmetic returns for all prices.  
You own 1 share of META. Remove the mean from the series so that the mean(META)=0  
Calculate VaR

In [9]:
# 计算 META 股票的每日算术收益
data = pd.read_csv('/Users/apple/Desktop/Duke/Fintech 545/Fintech-545/DailyPrices.csv')

# 'Date' 列为时间戳，'META' 列为 META 股票价格
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# 计算 META 的算术收益
data['META_Returns'] = data['META'].pct_change()  # 百分比变化（算术收益）
data.dropna(inplace=True)  # 删除 NaN

print(data.head())

                   SPY        AAPL        MSFT        AMZN        TSLA       GOOGL        GOOG        META       NVDA       BRK-B  ...       MDLZ         MO         ADI       GILD         LMT         SYK         GM        TFC        TJX  META_Returns
Date                                                                                                                               ...                                                                                                                    
2023-09-06  440.064606  181.978806  330.400024  135.360001  251.919998  134.127884  135.038361  298.556488  47.046768  361.670013  ...  68.004555  39.815193  179.055954  70.584099  412.773102  286.184906  32.462914  28.058586  89.469780     -0.003265
2023-09-07  438.713501  176.656036  327.452148  137.850006  251.490005  134.925903  135.866348  298.057526  46.227013  361.799988  ...  68.307068  39.988983  175.399750  70.298882  414.720398  286.105713  32.215633  27.633739  90.149963     -0.001

In [10]:
# 去除收益的均值，使均值(META_Returns) = 0
meta_returns_mean = data['META_Returns'].mean()
data['META_Adjusted'] = data['META_Returns'] - meta_returns_mean

# 检查调整后的均值是否为 0
meta_adjusted_mean = data['META_Adjusted'].mean()
print("Adjusted META_Returns Mean:", meta_adjusted_mean)

# 查看调整后的收益
print(data[['META_Returns', 'META_Adjusted']].head())

Adjusted META_Returns Mean: 2.2293635032633663e-19
            META_Returns  META_Adjusted
Date                                   
2023-09-06     -0.003265      -0.005754
2023-09-07     -0.001671      -0.004161
2023-09-08     -0.002612      -0.005101
2023-09-11      0.032462       0.029972
2023-09-12     -0.019183      -0.021673


**2.1. Using a normal distribution.**

In [11]:
def calculate_var_normal(returns, confidence_level=0.95):
    # 计算收益的均值和标准差
    mean = np.mean(returns)
    std_dev = np.std(returns)
    
    # 根据置信水平计算 VaR
    z_score = norm.ppf(1 - confidence_level)
    var = mean + z_score * std_dev
    return var

# 使用调整后的收益计算 VaR
var_normal = calculate_var_normal(data['META_Adjusted'], confidence_level=0.95)
print(f"VaR (Normal Distribution): {var_normal}")

VaR (Normal Distribution): -0.03817295454890712


**2.2 Using a normal distribution with an Exponentially Weighted variance (λ = 0. 94)**

In [12]:
def calculate_var_ewma(returns, lambda_=0.94, confidence_level=0.95):
    # 使用 EWMA 计算波动率
    weights = np.array([(1 - lambda_) * lambda_ ** i for i in range(len(returns))])
    weights = weights[::-1]  # 权重从当前往过去
    weighted_variance = np.sum(weights * returns ** 2)
    weighted_std_dev = np.sqrt(weighted_variance)

    # 计算 VaR
    z_score = norm.ppf(1 - confidence_level)
    var = z_score * weighted_std_dev
    return var

# 计算基于EWMA的VaR
var_ewma = calculate_var_ewma(data['META_Adjusted'], lambda_=0.94, confidence_level=0.95)
print(f"VaR (EWMA): {var_ewma}")

VaR (EWMA): -0.030991424222976006


**2.3 Using a MLE fitted T distribution.**

In [13]:
def calculate_var_t_distribution(returns, confidence_level=0.95):
    # 使用 MLE 拟合 t 分布
    params = t.fit(returns)
    df, loc, scale = params
    
    # 计算 t 分布的 VaR
    var = t.ppf(1 - confidence_level, df, loc=loc, scale=scale)
    return var

# 计算基于 T 分布的 VaR
var_t = calculate_var_t_distribution(data['META_Adjusted'], confidence_level=0.95)
print(f"VaR (T Distribution): {var_t}")

VaR (T Distribution): -0.03242585905836266


**2.4 Using a fitted AR(1) model.**

In [14]:
def calculate_var_ar1(returns, confidence_level=0.95):
    # 拟合 AR(1) 模型
    model = AutoReg(returns, lags=1)
    ar1_fit = model.fit()
    
    # 预测未来收益
    forecast = ar1_fit.predict(start=len(returns), end=len(returns))
    predicted_return = forecast.iloc[0]  # 使用 .iloc[0] 获取预测的第一个值
    
    # 计算标准差（残差的标准差）
    residual_std_dev = np.std(ar1_fit.resid)
    
    # 计算 VaR
    z_score = norm.ppf(1 - confidence_level)
    var = predicted_return + z_score * residual_std_dev
    return var

# 计算基于 AR(1) 的 VaR
var_ar1 = calculate_var_ar1(data['META_Adjusted'], confidence_level=0.95)
print(f"VaR (AR(1) Model: {var_ar1}")

VaR (AR(1) Model: -0.03837900413254759


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/tsa/deterministic.py:308: UserWarning: 

**2.5 Using a Historic Simulation.**

In [15]:
def calculate_var_historical(returns, confidence_level=0.95):
    # 根据历史数据计算分位数
    var = np.percentile(returns, (1 - confidence_level) * 100)
    return var

# 计算基于历史模拟的 VaR
var_historical = calculate_var_historical(data['META_Adjusted'], confidence_level=0.95)
print(f"VaR (Historical Simulation): {var_historical}")

VaR (Historical Simulation): -0.02884348903974016


**Compare the 5 values.**

In [16]:
# 打印和比较 VaR 结果
print(f"VaR Comparison:")
print(f"1. Normal Distribution: {var_normal}")
print(f"2. EWMA: {var_ewma}")
print(f"3. T Distribution: {var_t}")
print(f"4. AR(1) Model: {var_ar1}")
print(f"5. Historical Simulation: {var_historical}")

VaR Comparison:
1. Normal Distribution: -0.03817295454890712
2. EWMA: -0.030991424222976006
3. T Distribution: -0.03242585905836266
4. AR(1) Model: -0.03837900413254759
5. Historical Simulation: -0.02884348903974016


正态分布假设收益分布是对称的，且没有fat tails，因此 VaR 的估计通常较小(-0.0381).  
通过 EWMA 计算历史波动率，赋予最近的收益更高的权重（λ=0.94）。由于对近期数据赋予了更大的权重，EWMA 更敏感于最新的市场波动，因此 VaR 的值比正态分布小了一些（-3.09%）  
T分布用于处理有厚尾的收益分布。相比于正态分布，T 分布能够更好地捕捉极端事件的风险。因此，VaR 的值介于正态分布和 EWMA 之间(-3.24%)  
AR(1) 模型捕捉时间序列中的自相关性，基于过去的数据预测未来的收益。如果市场具有较强的自相关性，AR(1) 模型可能会显著影响预测结果。然而，在这里AR(1) 模型给出的 VaR 值与正态分布非常相近，这表明市场收益中的自相关性可能较弱，或者收益的波动性没有显著的自我关联。
历史模拟法直接使用过去的历史收益数据进行计算，不假设收益分布的形式。这里基于历史数据估算的 VaR 为 -2.88%，这是五种方法中 VaR 值最小的一个，这可能是因为最近的市场表现较为平稳。

### Problem 3
Using Portfolio.csv and DailyPrices.csv. Assume the expected return on all stocks is 0.  
This file contains the stock holdings of 3 portfolios. You own each of these portfolios. Using an exponentially weighted covariance with lambda = 0.97, calculate the VaR of each portfolio as well as your total VaR (VaR of the total holdings). Express VaR as a $.  

**3.1 Discuss your methods and your results.**

In [22]:
# 两个文件的股票差异
# 读取投资组合持仓数据
portfolio_data = pd.read_csv('/Users/apple/Desktop/Duke/Fintech 545/Fintech-545/portfolio.csv')

# 读取每日股票价格数据
prices_data = pd.read_csv('/Users/apple/Desktop/Duke/Fintech 545/Fintech-545/DailyPrices.csv')

# 提取 Portfolio.csv 中的股票名称
portfolio_stocks = portfolio_data['Stock'].unique()

# 提取 DailyPrices.csv 中的股票名称（假设股票名在列名中，除去日期列）
prices_stocks = prices_data.columns.drop('Date')

# 打印 Portfolio.csv 中的股票名称
print("Stocks in Portfolio.csv:")
print(portfolio_stocks)

# 打印 DailyPrices.csv 中的股票名称
print("\nStocks in DailyPrices.csv:")
print(prices_stocks)

# 计算并打印两个文件中的股票差异
missing_in_prices = set(portfolio_stocks) - set(prices_stocks)
missing_in_portfolio = set(prices_stocks) - set(portfolio_stocks)

print("\nStocks in Portfolio.csv but not in DailyPrices.csv:")
print(missing_in_prices)

print("\nStocks in DailyPrices.csv but not in Portfolio.csv:")
print(missing_in_portfolio)


Stocks in Portfolio.csv:
['AAPL' 'MSFT' 'AMZN' 'NVDA' 'GOOGL' 'TSLA' 'GOOG' 'BRK-B' 'META' 'UNH'
 'XOM' 'LLY' 'JPM' 'JNJ' 'V' 'PG' 'MA' 'AVGO' 'HD' 'CVX' 'MRK' 'ABBV'
 'COST' 'PEP' 'ADBE' 'WMT' 'KO' 'CSCO' 'CRM' 'MCD' 'ACN' 'BAC' 'TMO'
 'CMCSA' 'PFE' 'LIN' 'ORCL' 'ABT' 'NFLX' 'DHR' 'AMD' 'WFC' 'DIS' 'PM'
 'TXN' 'INTC' 'AMGN' 'COP' 'INTU' 'VZ' 'CAT' 'NEE' 'IBM' 'UNP' 'HON' 'BMY'
 'LOW' 'GE' 'SPGI' 'QCOM' 'AMAT' 'BA' 'UPS' 'NKE' 'NOW' 'T' 'GS' 'BKNG'
 'MS' 'MDT' 'SBUX' 'ELV' 'PLD' 'RTX' 'DE' 'TJX' 'ISRG' 'ADP' 'MMC' 'MDLZ'
 'SYK' 'BLK' 'GILD' 'LMT' 'CVS' 'VRTX' 'AXP' 'REGN' 'CB' 'ADI' 'CI' 'ETN'
 'SLB' 'PGR' 'SCHW' 'LRCX' 'ZTS' 'C' 'BSX' 'AMT']

Stocks in DailyPrices.csv:
Index(['SPY', 'AAPL', 'MSFT', 'AMZN', 'TSLA', 'GOOGL', 'GOOG', 'META', 'NVDA', 'BRK-B', 'JPM', 'JNJ', 'UNH', 'HD', 'PG', 'V', 'BAC', 'MA', 'PFE', 'XOM', 'DIS', 'CSCO', 'AVGO', 'ADBE', 'CVX', 'PEP', 'TMO', 'KO', 'ABBV', 'CMCSA', 'NFLX', 'ABT', 'ACN', 'COST', 'CRM', 'INTC', 'WFC', 'VZ', 'PYPL', 'WMT', 'QCOM', 'MRK', 'LLY'

——————————————————————————————————————————

In [24]:
# 读取投资组合持仓数据
portfolio_data = pd.read_csv('/Users/apple/Desktop/Duke/Fintech 545/Fintech-545/portfolio.csv')

# 读取每日股票价格数据
prices_data = pd.read_csv('/Users/apple/Desktop/Duke/Fintech 545/Fintech-545/DailyPrices.csv')

# 打印两者的前几行，确保数据加载正确
print(portfolio_data.head())
print(prices_data.head())

# 处理日期列，假设日期列名为 'Date'
prices_data['Date'] = pd.to_datetime(prices_data['Date'])
prices_data.set_index('Date', inplace=True)

# 打印处理后的价格数据以确认
print(prices_data.head())


  Portfolio  Stock  Holding
0         A   AAPL      158
1         A   MSFT      178
2         A   AMZN      110
3         A   NVDA       54
4         A  GOOGL       69
                  Date         SPY        AAPL        MSFT        AMZN        TSLA       GOOGL        GOOG        META       NVDA  ...         PNC       MDLZ         MO         ADI       GILD         LMT         SYK         GM        TFC        TJX
0  2023-09-05 00:00:00  443.042969  188.734238  331.065002  137.270004  256.489990  135.434647  136.375107  299.534485  48.529320  ...  115.410866  67.877701  40.327404  178.426926  71.458801  433.463379  283.403229  32.917908  28.738337  90.317558
1  2023-09-06 00:00:00  440.064606  181.978806  330.400024  135.360001  251.919998  134.127884  135.038361  298.556488  47.046768  ...  113.370651  68.004555  39.815193  179.055954  70.584099  412.773102  286.184906  32.462914  28.058586  89.469780
2  2023-09-07 00:00:00  438.713501  176.656036  327.452148  137.850006  251.490005  1

In [25]:
# 提取 Portfolio.csv 中的股票
portfolio_stocks = portfolio_data['Stock'].unique()

# 提取 DailyPrices.csv 中的股票（列名）
prices_stocks = prices_data.columns

# 找出两者共有的股票
common_stocks = set(portfolio_stocks).intersection(prices_stocks)

# 如果找不到共有股票，停止执行并输出信息
if len(common_stocks) == 0:
    print("No common stocks found!")
else:
    print(f"Common stocks: {common_stocks}")

# 将 common_stocks 转换为 list
common_stocks_list = list(common_stocks)

# 过滤 Portfolio.csv 和 DailyPrices.csv 中共有的股票
portfolio_filtered = portfolio_data[portfolio_data['Stock'].isin(common_stocks_list)]
prices_filtered = prices_data[common_stocks_list]

# 查看过滤后的数据
print(portfolio_filtered.head())
print(prices_filtered.head())


Common stocks: {'SCHW', 'ZTS', 'LRCX', 'TJX', 'CVX', 'V', 'GOOGL', 'VZ', 'RTX', 'NKE', 'MRK', 'AXP', 'BRK-B', 'PLD', 'AMD', 'TMO', 'JNJ', 'JPM', 'GS', 'LLY', 'COST', 'META', 'PG', 'PFE', 'HD', 'SBUX', 'MDT', 'INTU', 'NOW', 'PEP', 'CRM', 'ABT', 'GOOG', 'WMT', 'UNH', 'NVDA', 'CAT', 'LIN', 'LMT', 'ACN', 'ABBV', 'UNP', 'NEE', 'MS', 'DHR', 'CMCSA', 'WFC', 'AMAT', 'ADI', 'CSCO', 'BAC', 'MDLZ', 'MSFT', 'AVGO', 'ADBE', 'ADP', 'COP', 'ISRG', 'BLK', 'DE', 'AMGN', 'UPS', 'C', 'SYK', 'LOW', 'BKNG', 'T', 'IBM', 'QCOM', 'GILD', 'NFLX', 'XOM', 'DIS', 'BMY', 'TSLA', 'MA', 'MCD', 'TXN', 'HON', 'BA', 'INTC', 'SPGI', 'CVS', 'ORCL', 'AMT', 'GE', 'AAPL', 'PM', 'AMZN', 'KO'}
  Portfolio  Stock  Holding
0         A   AAPL      158
1         A   MSFT      178
2         A   AMZN      110
3         A   NVDA       54
4         A  GOOGL       69
                 SCHW         ZTS        LRCX        TJX         CVX           V       GOOGL         VZ        RTX        NKE  ...       INTC        SPGI        CVS      

In [26]:
# 计算每日股票收益率
# 计算每日收益率（假设预期收益为0）
returns = prices_filtered.pct_change().dropna()

# 查看每日收益率
print(returns.head())

                SCHW       ZTS      LRCX       TJX       CVX         V     GOOGL        VZ       RTX       NKE  ...      INTC      SPGI       CVS      ORCL       AMT        GE      AAPL        PM      AMZN        KO
Date                                                                                                            ...                                                                                                    
2023-09-06 -0.023626 -0.018747 -0.006464 -0.009387  0.001021  0.003342 -0.009649 -0.003499 -0.018107 -0.001395  ...  0.007355  0.001525 -0.001524  0.002823 -0.013358 -0.001772 -0.035793 -0.010158 -0.013914 -0.000680
2023-09-07 -0.007723  0.004923 -0.028065  0.007602  0.000060  0.003981  0.005950 -0.010825  0.009819 -0.022460  ...  0.032450 -0.001446  0.001374  0.006113  0.025504 -0.007898 -0.029249 -0.002886  0.018395 -0.007656
2023-09-08  0.026116 -0.010651 -0.011556  0.000000  0.003420  0.000607  0.008280 -0.010648 -0.010079 -0.002655  ... -0.004453 -0.007292 

In [27]:
# 使用 λ = 0.97 计算协方差矩阵
# 设置指数加权的衰减系数 lambda = 0.97
lambda_ = 0.97

# 使用 EWMA 计算协方差矩阵
cov_matrix_ewma = returns.ewm(span=(1 / (1 - lambda_))).cov(pairwise=True)

# 获取最新时间点的协方差矩阵
cov_matrix_latest = cov_matrix_ewma.loc[returns.index[-1]]

# 查看协方差矩阵
print(cov_matrix_latest)

          SCHW       ZTS      LRCX       TJX       CVX         V     GOOGL        VZ       RTX       NKE  ...      INTC      SPGI       CVS      ORCL       AMT        GE      AAPL        PM      AMZN        KO
SCHW  0.000292 -0.000010  0.000160 -0.000028  0.000047  0.000036  0.000021 -0.000027  0.000048  0.000036  ...  0.000143  0.000039  0.000015  0.000086 -0.000015  0.000067  0.000051 -0.000016  0.000071 -0.000018
ZTS  -0.000010  0.000200  0.000040  0.000048 -0.000007  0.000038  0.000057  0.000026  0.000009  0.000107  ...  0.000187  0.000062  0.000057  0.000027  0.000021  0.000058  0.000017  0.000060  0.000074  0.000020
LRCX  0.000160  0.000040  0.001175  0.000154  0.000138  0.000092  0.000227 -0.000073  0.000004  0.000248  ...  0.001209  0.000082  0.000048  0.000316 -0.000210  0.000311  0.000163 -0.000058  0.000504 -0.000121
TJX  -0.000028  0.000048  0.000154  0.000263  0.000035  0.000048  0.000083  0.000017 -0.000019  0.000111  ...  0.000208  0.000044  0.000059  0.000079  0.000018 

In [28]:
# 计算组合的 VaR
# 计算组合权重的函数
def calculate_portfolio_weights(portfolio):
    total_holding = portfolio['Holding'].sum()
    portfolio_weights = portfolio.set_index('Stock')['Holding'] / total_holding
    return portfolio_weights

# 根据协方差矩阵计算 VaR
def calculate_portfolio_var(portfolio, cov_matrix, confidence_level=0.95):
    # 计算组合权重
    portfolio_weights = calculate_portfolio_weights(portfolio)

    # 获取协方差矩阵的子集
    stocks_in_portfolio = portfolio['Stock'].values
    cov_matrix_filtered = cov_matrix.loc[stocks_in_portfolio, stocks_in_portfolio]

    # 确保权重和协方差矩阵股票顺序一致
    portfolio_weights = portfolio_weights.reindex(cov_matrix_filtered.index)

    # 计算组合的方差
    portfolio_variance = np.dot(np.dot(portfolio_weights.T, cov_matrix_filtered), portfolio_weights)

    # 获取 z 分位数
    z_score = norm.ppf(1 - confidence_level)

    # 计算 VaR
    portfolio_var = z_score * np.sqrt(portfolio_variance)

    return portfolio_var

# 计算每个组合的 VaR
portfolio_groups = portfolio_filtered.groupby('Portfolio')

# 遍历每个组合并计算 VaR
for portfolio_name, portfolio_group in portfolio_groups:
    portfolio_var = calculate_portfolio_var(portfolio_group, cov_matrix_latest)
    print(f"VaR for Portfolio {portfolio_name}: {portfolio_var}")


VaR for Portfolio A: -0.014524653541482755
VaR for Portfolio B: -0.016156808631077147
VaR for Portfolio C: -0.01470369493827952


In [29]:
# 将 VaR 转换为美元值
# 根据协方差矩阵计算 VaR（以美元为单位）
def calculate_portfolio_var_in_dollars(portfolio, cov_matrix, prices_data, confidence_level=0.95):
    # 计算组合权重
    portfolio_weights = calculate_portfolio_weights(portfolio)

    # 获取协方差矩阵的子集
    stocks_in_portfolio = portfolio['Stock'].values
    cov_matrix_filtered = cov_matrix.loc[stocks_in_portfolio, stocks_in_portfolio]

    # 确保权重和协方差矩阵股票顺序一致
    portfolio_weights = portfolio_weights.reindex(cov_matrix_filtered.index)

    # 计算组合的方差
    portfolio_variance = np.dot(np.dot(portfolio_weights.T, cov_matrix_filtered), portfolio_weights)

    # 获取 z 分位数
    z_score = norm.ppf(1 - confidence_level)

    # 计算 VaR 相对值
    portfolio_var_relative = z_score * np.sqrt(portfolio_variance)

    # 计算组合的总市值（基于持有数量和最新价格）
    portfolio['Price'] = prices_data.loc[prices_data.index[-1], portfolio['Stock']].values  # 获取最新价格
    portfolio['Value'] = portfolio['Holding'] * portfolio['Price']  # 计算每个股票的市值
    total_portfolio_value = portfolio['Value'].sum()  # 总市值

    # 计算 VaR 以美元为单位
    portfolio_var_dollar = total_portfolio_value * portfolio_var_relative

    return portfolio_var_dollar


In [30]:
# 计算每个组合的 VaR（以美元为单位）
for portfolio_name, portfolio_group in portfolio_groups:
    portfolio_var_dollar = calculate_portfolio_var_in_dollars(portfolio_group, cov_matrix_latest, prices_filtered)
    print(f"VaR for Portfolio {portfolio_name} (in $): {abs(portfolio_var_dollar):.2f}")

VaR for Portfolio A (in $): 18435.70
VaR for Portfolio B (in $): 11599.78
VaR for Portfolio C (in $): 18080.01


In [31]:
# 计算所有组合的总 VaR
total_var_dollar = 0

# 遍历每个组合并汇总 VaR
for portfolio_name, portfolio_group in portfolio_groups:
    portfolio_var_dollar = calculate_portfolio_var_in_dollars(portfolio_group, cov_matrix_latest, prices_filtered)
    total_var_dollar += portfolio_var_dollar

print(f"Total EWMR VaR for all portfolios (in $): {abs(total_var_dollar):.2f}")

Total EWMR VaR for all portfolios (in $): 48115.49


**3.2.1 Choose a different model for returns and calculate VaR again.**

In [32]:
# 历史模拟法计算 VaR
def calculate_historical_var(returns, confidence_level=0.95):
    # 将收益率排序
    sorted_returns = returns.sort_values()

    # 根据置信水平找到对应的分位数
    var_percentile = int((1 - confidence_level) * len(sorted_returns))

    # VaR 是这个分位数对应的值
    historical_var = sorted_returns.iloc[var_percentile]

    return abs(historical_var)  # 取绝对值，表示损失

# 计算每个组合的历史 VaR
for portfolio_name, portfolio_group in portfolio_groups:
    # 获取组合中每只股票的收益率
    portfolio_returns = returns[portfolio_group['Stock'].values].mean(axis=1)  # 平均收益率
    portfolio_historical_var = calculate_historical_var(portfolio_returns)
    
    print(f"Historical VaR for Portfolio {portfolio_name}: {portfolio_historical_var:.4f}")

Historical VaR for Portfolio A: 0.0122
Historical VaR for Portfolio B: 0.0148
Historical VaR for Portfolio C: 0.0129


In [33]:
# 历史模拟法计算 VaR（以美元为单位）
def calculate_historical_var_in_dollars(portfolio, prices_data, returns, confidence_level=0.95):
    # 获取组合中每只股票的平均收益率
    portfolio_returns = returns[portfolio['Stock'].values].mean(axis=1)  # 平均收益率

    # 计算历史 VaR
    historical_var_relative = calculate_historical_var(portfolio_returns, confidence_level)

    # 计算组合的总市值（基于持有数量和最新价格）
    portfolio['Price'] = prices_data.loc[prices_data.index[-1], portfolio['Stock']].values  # 获取最新价格
    portfolio['Value'] = portfolio['Holding'] * portfolio['Price']  # 计算每个股票的市值
    total_portfolio_value = portfolio['Value'].sum()  # 总市值

    # 将历史 VaR 转换为美元
    portfolio_historical_var_dollar = total_portfolio_value * historical_var_relative

    return portfolio_historical_var_dollar

# 计算每个组合的历史 VaR（以美元为单位）
for portfolio_name, portfolio_group in portfolio_groups:
    portfolio_historical_var_dollar = calculate_historical_var_in_dollars(portfolio_group, prices_filtered, returns)
    print(f"Historical VaR for Portfolio {portfolio_name} (in $): {abs(portfolio_historical_var_dollar):.2f}")


Historical VaR for Portfolio A (in $): 15483.26
Historical VaR for Portfolio B (in $): 10610.18
Historical VaR for Portfolio C (in $): 15812.37


In [34]:
# 计算所有组合的总历史 VaR
total_historical_var_dollar = 0

# 遍历每个组合并汇总历史 VaR
for portfolio_name, portfolio_group in portfolio_groups:
    portfolio_historical_var_dollar = calculate_historical_var_in_dollars(portfolio_group, prices_filtered, returns)
    total_historical_var_dollar += portfolio_historical_var_dollar

# 输出所有组合的总历史 VaR
print(f"Total Historical VaR for all portfolios (in $): {abs(total_historical_var_dollar):.2f}")

Total Historical VaR for all portfolios (in $): 41905.81


**3.2.2 Why did you choose that model?**

选择历史模拟法的原因：  
历史模拟法不依赖正态分布假设，它直接使用历史数据，能够捕捉到市场中的极端波动。  
当市场出现即极端事件时，历史模拟法更能反映潜在的风险。  
比较两者：  
EWMA VaR 更适合当前市场环境的平滑风险评估，它对于捕捉最近的波动性变化更敏感。  
历史 VaR 则能够更好地捕捉历史上发生的极端事件，因而在某些情况下它的 VaR 会更高，尤其是在高波动市场中。


**3.3 How did the model change affect the results?**

**EWMA VaR 值更高**：
   - **EWMA 模型**在组合中给出了更高的VaR值$48,115.49。由于 EWMA 对最近的市场波动给予了更高的权重，这说明基于当前市场的波动性更大，EWMA 认为组合有更大的短期风险。

**历史模拟 VaR 值较低**：
   - **历史模拟法**的 VaR 值为$41,905.81，低于 EWMA VaR。历史模拟法不会过度放大最近的市场波动，而是基于长时间的历史数据，这个结果表明，从历史数据来看，过去的市场波动性可能没有当前那么剧烈。

总的来说，**EWMA 模型**更强调当前市场环境，反映短期内的风险。**历史模拟法**更侧重历史数据，适合用于评估长期的风险，尤其是捕捉尾部风险（即极端事件的可能性）。因此，当我们考虑组合在长期中的表现时，历史模拟法提供了一个更稳定的视角。